In [1]:
%env TORCH_USE_CUDA_DSA=1
%env CUDA_LAUNCH_BLOCKING=1

env: TORCH_USE_CUDA_DSA=1
env: CUDA_LAUNCH_BLOCKING=1


In [1]:
from typing import Tuple
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

import numpy as np

import torch

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Optimizer, Adam
from torch.optim.lr_scheduler import LRScheduler

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
torch.set_default_dtype(torch.float64)

In [4]:
test_df = pd.read_pickle("test_proc.pkl")
test_df

,0,1,2,3,4,5,6,7,8,9,...,1000,element_1,element_2,element_3,element_1_ratio,element_2_ratio,element_3_ratio,temp,pressure,air_ratio
51976,1.429725e-20,3.325318e-20,1.513897e-20,1.119836e-21,1.209848e-21,2.024112e-21,7.172782e-21,4.953834e-21,1.539859e-21,1.285451e-21,...,9.105786e-25,9,12,22,0.136030,0.262144,0.601825,283.0,0.2,0.6
18970,5.940243e-23,7.336601e-23,9.720283e-23,1.447083e-22,2.680262e-22,8.343595e-22,1.010416e-20,6.042984e-21,7.479574e-22,3.643073e-22,...,1.465311e-22,0,21,22,0.188355,0.316809,0.494835,283.0,0.1,0.6
30497,1.193615e-20,6.945339e-21,6.543001e-22,2.529143e-22,1.749403e-22,3.243140e-22,3.550860e-22,1.091960e-22,9.629268e-23,1.882503e-22,...,7.445923e-23,5,16,19,0.444071,0.477905,0.078024,323.0,0.1,0.6
30356,1.624151e-20,5.577240e-21,9.267746e-22,2.168774e-21,5.549700e-21,4.163047e-21,1.670818e-21,8.912111e-22,8.374945e-22,1.444200e-21,...,3.604447e-22,4,21,24,0.509421,0.224513,0.266066,323.0,0.1,0.6
140548,2.003258e-21,1.077312e-21,3.272797e-22,1.452913e-22,9.167639e-23,6.963908e-23,5.832702e-23,5.251748e-23,4.962818e-23,4.851140e-23,...,9.615524e-22,4,13,18,0.227958,0.482563,0.289478,263.0,0.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218031,3.535618e-22,4.722553e-22,4.692500e-22,5.307244e-22,6.048017e-22,6.291295e-22,6.453254e-22,6.485575e-22,7.700791e-22,1.061828e-21,...,8.640137e-25,6,9,-1,0.530767,0.469233,0.000000,323.0,0.8,0.6
57587,9.397569e-22,1.173034e-21,2.279960e-21,8.301202e-22,8.418463e-22,9.303921e-22,1.093095e-21,2.257390e-21,3.849821e-21,3.221007e-21,...,1.020158e-21,14,15,18,0.600135,0.375171,0.024694,303.0,0.2,0.6
254241,7.204798e-22,1.141918e-21,2.021069e-21,3.898292e-21,4.167931e-21,3.322392e-21,2.260052e-21,1.356805e-21,9.869838e-22,1.119172e-21,...,1.174975e-22,10,18,20,0.273032,0.469203,0.257765,293.0,1.0,0.6
30894,4.030195e-22,5.592868e-22,9.234086e-22,2.112966e-21,5.189306e-21,3.964872e-21,1.755474e-21,1.090197e-21,1.088081e-21,1.695364e-21,...,1.690917e-23,8,14,24,0.375711,0.384662,0.239627,323.0,0.1,0.6


In [9]:
print(len(df))
df = df[(df[[str(i) for i in range(1001)]] > 0).all(axis=1)]
print(len(df))

183553
183553


In [10]:
len(df[(df["element_1"] == 0) | (df["element_2"] == 0) | (df["element_3"] == 0)]) / len(df)

0.11451733286843582

In [5]:
class CustomSpectraDataset(Dataset):
    def __init__(self, data: pd.DataFrame, device="cuda:0") -> None:
        self.data = data
        self.elements = self.data["element_1"].unique()
        self.air_ratios = data.air_ratio.to_numpy(dtype=np.float64)

        self.spectras = torch.log(
            torch.tensor(
                self.data[[str(i) for i in range(1001)]].to_numpy(dtype=np.float64)
            )
        ).to(device)

        self.ratios = torch.tensor(
            self.data[
                ["element_1_ratio", "element_2_ratio", "element_3_ratio"]
            ].to_numpy(dtype=np.float64)
        ).to(device)

        self.element_indices = self.data[
            ["element_1", "element_2", "element_3"]
        ].to_numpy(dtype=np.float64)

        self.elements_distributions = torch.zeros(
            [len(self.data), len(self.elements) + 1], dtype=torch.float64
        ).to(device)

        for idx in range(len(self.data)):
            indices = self.element_indices[idx, :]
            indices = indices[indices != -1]
            self.elements_distributions[idx, indices] = self.ratios[idx][
                range(indices.shape[0])
            ] * (1 - self.air_ratios[idx])
            self.elements_distributions[idx, -1] = self.air_ratios[idx]

        self.elements_distributions = self.elements_distributions[
            ~torch.isnan(self.spectras).any(dim=1)
        ]
        self.spectras = self.spectras[~torch.isnan(self.spectras).any(dim=1)]

    def __len__(self) -> int:
        return len(self.spectras)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        spectra = self.spectras[idx]
        elements_distribution = self.elements_distributions[idx]

        return spectra, elements_distribution

In [6]:
test_dataset = CustomSpectraDataset(test_df)

In [7]:
val_loader = DataLoader(
    test_dataset,
    batch_size=2048,
)

In [8]:
class THzFCNN(nn.Module):
    def __init__(self, n_elements: int) -> None:
        super().__init__()
        self.n_elements = n_elements
        self.net = nn.Sequential(
            nn.BatchNorm1d(1001),
            nn.Linear(in_features=1001, out_features=4000),
            nn.LeakyReLU(),
            nn.BatchNorm1d(4000),
            nn.Linear(in_features=4000, out_features=2000),
            nn.LeakyReLU(),
            nn.BatchNorm1d(2000),
            nn.Linear(in_features=2000, out_features=2000),
            nn.LeakyReLU(),
            nn.BatchNorm1d(2000),
            nn.Linear(in_features=2000, out_features=1000),
            nn.LeakyReLU(),
            nn.BatchNorm1d(1000),
            nn.Linear(in_features=1000, out_features=500),
            nn.LeakyReLU(),
            nn.BatchNorm1d(500),
            nn.Linear(in_features=500, out_features=500),
            nn.LeakyReLU(),
            nn.BatchNorm1d(500),
            nn.Linear(in_features=500, out_features=250),
            nn.LeakyReLU(),
            nn.BatchNorm1d(250),
            nn.Linear(in_features=250, out_features=250),
            nn.LeakyReLU(),
            nn.BatchNorm1d(250),
            nn.Linear(in_features=250, out_features=self.n_elements),
            nn.Dropout(p=0.05),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

In [9]:
net = THzFCNN(test_dataset[0][1].shape[0])
net.load_state_dict(torch.load("./fcnn-bs-2048-add-layer-small-dropout.model"))

net.to(device)


print(device)


pred, y_test = np.empty((0, 26)), np.empty((0, 26))
for spectra, target in tqdm(val_loader, desc="testing"):
    ans = net(spectra)
    ans = nn.Softmax()(ans).cpu().detach().numpy()
    pred = np.append(pred, ans, axis=0)
    y_test = np.append(y_test, target.cpu().numpy(), axis=0)

cuda:0


testing:   0%|          | 0/64 [00:00<?, ?it/s]C:\Users\whoee\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
testing: 100%|██████████| 64/64 [00:32<00:00,  1.97it/s]


In [18]:
for spectra, target in tqdm(val_loader, desc="testing"):
    ans = net(spectra)
    ans = nn.Softmax()(ans)
    break
ans[5], target[5]

testing:   0%|          | 0/64 [00:00<?, ?it/s]


(tensor([1.2818e-03, 2.9335e-02, 1.0418e-02, 9.4013e-04, 8.1593e-04, 7.5845e-04,
         7.4720e-04, 1.9152e-03, 1.0439e-02, 1.3736e-03, 1.1436e-03, 4.9432e-03,
         1.4880e-03, 1.5715e-03, 6.9828e-04, 2.1599e-03, 1.0729e-03, 1.3185e-01,
         9.2884e-04, 1.5478e-03, 6.8504e-04, 9.7989e-04, 1.1288e-03, 8.7979e-04,
         1.2551e-03, 7.8964e-01], device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0740,
         0.0000, 0.0000, 0.1145, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2114,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6000],
        device='cuda:0'))

In [11]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    mean_absolute_error,
)

In [38]:

print(f'F1: {f1_score(y_test, np.where(pred > 0.05, 1, 0), average="macro")}')
print(
    f'precision: {precision_score(y_test, np.where(pred > 0.1, 1, 0), average="macro")}'
)
print(f'recall: {recall_score(y_test, np.where(pred > 0.1, 1, 0), average="macro")}')
print(f"accuracy: {accuracy_score(y_test, np.where(pred > 0.1, 1, 0))}")

F1: 0.9082083020753646
precision: 0.9340409467676765
recall: 0.8836817939245001
accuracy: 0.619078872057084


In [39]:
print(
    f'precision: {precision_score(y_test, np.where(pred > 0.1, 1, 0), average=None)}'
)
print(f'recall: {recall_score(y_test, np.where(pred > 0.1, 1, 0), average=None)}')
print(f"F1: {f1_score(y_test, np.where(pred > 0.05, 1, 0), average=None)}")

precision: [0.99986054 0.49804203 0.50049659 0.99978677 0.99992991 0.99817953
 0.96094152 0.99712437 0.44192958 0.99113612 0.99829376 0.99270487
 1.         0.99992889 0.99964664 0.99663276 0.99992883 0.99993001
 0.99986008 1.         0.995998   0.99971595 0.98542831 0.99852092
 0.99700769]
recall: [0.94734408 0.51142685 0.50561873 0.93829631 0.94620946 0.9490713
 0.86798219 0.91690355 0.51239231 0.92690789 0.93246564 0.93055371
 0.9432808  0.94287247 0.94520548 0.9436732  0.94307961 0.94665695
 0.94436368 0.94223081 0.92267461 0.94053982 0.93102987 0.94024406
 0.92102146]
F1: [0.97332113 0.5151857  0.51233036 0.96929312 0.97286439 0.97259574
 0.88562124 0.95503855 0.49646586 0.95332371 0.96494522 0.96032151
 0.97177033 0.97187758 0.9724242  0.96855432 0.97126298 0.97347661
 0.97226654 0.97111728 0.96032533 0.96899438 0.94664686 0.9696268
 0.95555781]


In [28]:
np.array(list(map(float, "0.97332113 0.5151857 0.51233036 0.96929312 0.97286439 0.97259574 0.88562124 0.95503855 0.49646586 0.95332371 0.96494522 0.96032151 0.97177033 0.97187758 0.9724242 0.96855432 0.97126298 0.97347661 0.97226654 0.97111728 0.96032533 0.96899438 0.94664686 0.9696268 0.95555781".split(" ")))).mean()

0.908208302

In [13]:
def cross_entropy(predictions, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions.
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1.0 - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets * np.log(predictions + 1e-9)) / N
    return ce

In [15]:
print(f'MAE: {mean_absolute_error(y_test, pred)}')
print(f"MAE: {mean_absolute_error(y_test, pred, multioutput='raw_values')}")
print(f"Cross Entropy: {cross_entropy(y_test, pred)}")

MAE: 0.029936199440300314
MAE: [0.01662792 0.02609384 0.02538093 0.0129612  0.01923808 0.01695865
 0.01805305 0.01919187 0.02949075 0.01551713 0.01465912 0.01210849
 0.02211429 0.0195869  0.01433084 0.01838715 0.01593775 0.01757408
 0.01874721 0.01934471 0.0124655  0.01884126 0.01451602 0.01129428
 0.01129266 0.33762751]
Cross Entropy: 3.198969430457932
